<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# LinkedIn - List new profile content fans
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/Linkedin_Follow_number_of_content_published.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #html #plotly #csv #image #content #analytics #automation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

This notebook follows the evolution of content shares on LinkedIn.

## Input

In [ ]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

### Setup Variables

In [ ]:
KPI_TITLE = "New fans"
INPUT_DATABASE = LK_PROFILE_ENGAGEMENTS
OWNER = LK_FULLNAME
OWNER_ID = LK_PROFILE_ID
PLATFORM = LINKEDIN
LOGO = LINKEDIN_LOGO

### Setup Constants

In [ ]:
# Inputs
PERIOD = PERIOD_MTD

## Model

### Get your engagements

In [ ]:
df_engagements = get_data(INPUT_DATABASE)
print("✅ Engagements fetched:", len(df_engagements))
df_engagements.head(1)

### Get fans

In [ ]:
def get_fans(df):
    # Init
    df_fans = df.copy()
    
    # Filter
    df_fans = df_fans.drop_duplicates(["PROFILE_ID", "REACTION"], keep="last").reset_index(drop=True)
    df_fans["SCENARIO"] = pd.to_datetime(df_fans["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
    df_fans = df_fans[(df_fans["SCENARIO"] == datetime.now().strftime(PERIOD)) & (df_fans["FULLNAME"] != OWNER)].reset_index(drop=True)
    return df_fans

df_fans = get_fans(df_engagements)
df_fans

## Output

### Update Notion Report

In [ ]:
def update_report_status(report,
                         platform,
                         owner,
                         df_engagements,
                         df=pd.DataFrame(),
                         image_link=None,
                         html_link=None,
                         csv_link=None,
                         page_icon=LOGO,
                         ):
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
#         notion.connect(NOTION_TOKEN).blocks.delete(page.id)
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Check if image already exists
        blocks = page.get_blocks()
        if len(blocks) > 0:
            while True:
                for block in blocks:
                    notion.connect(NOTION_TOKEN).blocks.delete(block.id)
                blocks = page.get_blocks()
                if len(blocks) == 0:
                    break
        page.update()
        
        # Add new fans by posts
        df_posts = df_engagements[["TITLE", "POST_URL", "PUBLISHED_DATE"]].drop_duplicates().reset_index(drop=True)[:10]
        for index, row in df_posts.iterrows():
            # Add posts
            res = page.heading_3(row.TITLE)
            res.heading_3.text[0].href = row.POST_URL
            res.heading_3.text[0].text.link = Link(row.POST_URL)
            
            # Add new
            tmp_df = df[df.POST_URL == row.POST_URL].reset_index(drop=True)
            def add_reactions(df, reaction):
                df = df[df.REACTION == reaction]
                if reaction == "LIKES":
                    page.paragraph(f"New profile who liked ({len(df)}):")
                elif reaction == "COMMENTS":
                    page.paragraph(f"New profile who commented ({len(df)}):")
                for index, row in df.iterrows():
                    text = f"{row.FULLNAME} - {row.OCCUPATION}"
                    res = page.bulleted_list_item(text)
                    res.bulleted_list_item.text[0].href = row.PROFILE_URL
                    res.bulleted_list_item.text[0].text.link = Link(row.PROFILE_URL)
                page.paragraph("")
            add_reactions(tmp_df, "COMMENTS")
            add_reactions(tmp_df, "LIKES")


        # Create page in Notion
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)

update_report_status(KPI_TITLE,
                     PLATFORM,
                     OWNER,
                     df_engagements,
                     df_fans)